In [35]:
import pandas as pd

df = pd.read_csv("midi_motives.csv")
# df['initial_note'] = df['midi_notes'].apply(lambda x: x[1:3])

# # Group by direction and scale, and get the initial notes
# result = df.groupby(['direction', 'scale'])['initial_note'].apply(list).reset_index()

# print(result)

# Exclude rows where scale is "eas"
df_filtered = df[df['scale'] != 'eas']

# Extract the initial note from midi_notes for each combination of direction and scale
df_filtered['initial_note'] = df_filtered['midi_notes'].apply(lambda x: x[1:3])

# Group by direction and scale, and get the initial notes
result = df_filtered.groupby(['direction', 'scale'])['initial_note'].apply(list).reset_index()

# Save the results to JSON
result.to_json('initial_notes.json', orient='records')

print("The results have been saved to initial_notes.json")

The results have been saved to initial_notes.json


/tmp/ipykernel_77815/2670820685.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['initial_note'] = df_filtered['midi_notes'].apply(lambda x: x[1:3])


In [41]:
import pandas as pd
import json
import random

# Load the JSON file
with open('initial_notes.json', 'r') as file:
    data = json.load(file)

# Function to get initial notes for given direction and scale
def get_initial_notes(data, direction, scale):
    for entry in data:
        if entry['direction'] == direction and entry['scale'] == scale:
            return entry['initial_note']
    return None

# Given values for direction and scale
direction = 0
scale = 'min'

# Get the corresponding initial notes
initial_notes = get_initial_notes(data, direction, scale)

if initial_notes:
    # Randomly sample among the initial notes
    sampled_note = random.choice(initial_notes)
    print(f"Randomly sampled note: {sampled_note}")
else:
    print(f"No initial notes found for direction {direction} and scale {scale}")


Randomly sampled note: 56


In [7]:
import pandas as pd
import json
import random

def load_data(filepath):
    """Load the JSON data from a file."""
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

def get_initial_notes(data, direction, scale):
    """Get initial notes for given direction and scale from the data."""
    for entry in data:
        if entry['direction'] == direction and entry['scale'] == scale:
            return entry['initial_note']
    return None

def sample_initial_note(direction, scale):
    """Load data, find initial notes for given direction and scale, and randomly sample one note."""
    data = load_data('initial_notes.json')
    initial_notes = get_initial_notes(data, direction, scale)

    if initial_notes:
        sampled_note = random.choice(initial_notes)
        print(f"Randomly sampled note: {sampled_note}")
        return int(sampled_note)
    else:
        print(f"No initial notes found for direction {direction} and scale {scale}")
        return None

In [39]:
from markov_chain import MarkovManager
mm = MarkovManager()

foo = mm.get_model(scale="maj", trend="0")

sampled_note = sample_initial_note(direction=0, scale="maj")

start = [-1, sampled_note]
seq = foo.generate_sequence(start_notes=start, length=9)

seq[1:]

Randomly sampled note: 45
default note 45


[45, 53, 48, 57, 53, 60, 65, 67]

In [1]:
import pandas as pd
import numpy as np
from markov_chain import SecondOrderMarkovModel
import random
import ast

scales = ["maj", "min"]
trends = [0,1,2,3]
combinations = [(scale, trend) for scale in scales for trend in trends]

df = pd.read_csv("midi_motives.csv")

# Iterate through each combination
for scale, trend in combinations:
        
    # Filter the DataFrame for the current combination
    df_filtered = df[(df["direction"] == trend) & (df["scale"] == scale)]

    model = SecondOrderMarkovModel()
    model.train(df_filtered["midi_notes"])
    model.save_model(f"models/markov-{scale}-{trend}.pkl")


In [4]:
from markov_chain import SecondOrderMarkovModel

# Example usage

start_notes = [-1, 52]

new_model = SecondOrderMarkovModel()

trend = 0
scale = "maj"

new_model.load_model(f'models/markov-{scale}-{trend}.pkl')
new_generated_sequence = new_model.generate_sequence(start_notes, 10)
print(new_generated_sequence)


Warning, no transition avaliuable
[-1, 52, 53, 55, 60, 65, 72]
